# Exercise 2: Merging, Aggregating, Filtering, and Visualizing

In [ ]:
import altair as alt
import pandas as pd

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

* Read back in the `parquet` file with the `overall_score` you created from exercise 1.
* Read the Excel sheet containing the project information (scope of work, district, and project name).
* Use f-strings.

In [ ]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/starter_kit/"

In [ ]:
EXCEL_FILE = "starter_kit_csis_scoring_workbook.xlsx"

In [ ]:
OVERALL_SCORE_FILE = "starter_kit_example_final_scores.parquet"

In [ ]:
projects_df = pd.read_excel(f"{GCS_FILE_PATH}{EXCEL_FILE}")

In [ ]:
projects_df.head(2)

In [ ]:
overall_scores_df = pd.read_parquet(f"{GCS_FILE_PATH}{OVERALL_SCORE_FILE}")

In [ ]:
overall_scores_df.head(2)

## Merging 
* Your manager asks you to aggregate the dataframe by Caltrans District to find:
    * Median overall score
    * Max overall score 
    * Min overall score
    * Number of unique projects
* Annoyingly enough, the `overall_score` column and the `ct_district` are in two different dataframes. 
* You'll have to <b>merge</b> it on the common column(s) the two dataframes share.
* Welcome to DDS! This will happen to you all the time starting now. 

### Relevant Resources
* If needed, read about merges before diving in. 
    * [Resource #1 is a great tutorial for beginners](https://www.practicalpythonfordatascience.com/03_cleaning_data.html?highlight=merge#merging-dataframes-together).
    * [Resource #2 is written by our own Tiffany Ku, but it contains some geospatial references so it's a bit more to digest](https://docs.calitp.org/data-infra/analytics_new_analysts/01-data-analysis-intro.html#merge-tabular-and-geospatial-data-for-data-analysis).
### Food for thought 
* Which columns do the two dataframes have in common?
* What type of merge will achieve my goal?
    * Inner, outer, left, or right
* What do I expect out of the merge?
    * Do I expect all the values of the merge keys to be 1:1? Or m:1? 
    * Do I expect a project to correspond with multiple districts? Maybe, projects can and do cross multiple boundaries.
    * Do I expect a project to correspond with only one total cost estimate value? Yes, there shouldn't be multiple cost estimates for the same project!
* How do I go about checking the data after the merge?
    * Which arguments are available to help me per the [docs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html)?

In [ ]:
m1 = pd.merge(projects_df, overall_scores_df, on=["project_name"])

### Double Checking
* How many rows do you expect?
* How many unique projects are there? 
* <b>Hint</b>: check your original dataframes as well

In [ ]:
m1.shape

In [ ]:
m1.project_name.nunique()

In [ ]:
projects_df.project_name.nunique()

### The Beauty of Outer Joins 
* As you have noticed, we are missing a couple of projects.
* This is where `outer` joins are very useful.
* Merge your dataframes again using an `outer` join and with `indicator = True` on.
* Using `.value_counts()` check out how many rows are found in both dataframes, the left only, and the right only

In [ ]:
m2 = pd.merge(
    projects_df, overall_scores_df, on=["project_name"], indicator=True, how="outer"
)

In [ ]:
m2._merge.value_counts()

### Filtering
* Filter out for only the `left_only` and `right_only` values.
    * `!=` means does not equal to. 
    

In [ ]:
m2.loc[m2._merge != "both"][["project_name", "_merge"]]

* You could also use `isin([list of elements you want to keep])`

In [ ]:
m2.loc[m2._merge.isin(["left_only","right_only"])][["project_name", "_merge"]]

### Dictionaries
* String data is often entered in many different ways. BART can be entered in as bart, Bay Area Rapid Transit, BaRT, and more. 
* Often, strings are the reason why your dataframe is not merging properly.
* In Excel, it's easy to go in and manually tweak everything. However, that is not reproducible and time consuming. 
* Since there are essentially only a couple of names to replace, we can do it using a <b>dictionary</b>.

#### What is a dictionary?
* Per Practical Python for Data Science, a dictionary is <i>Dictionaries are used to store data values in key:value pairs. Similar to the list, a dictionary is a collection of objects. It is also mutable, meaning that you can add, remove, change values inside of it...With the list, we access elements using the index. With the dictionary, we access elements using keys.</i>.
    * Read more [here](https://www.practicalpythonfordatascience.com/00_python_crash_course_datatypes.html?highlight=dictionary#dictionary) and experiment with the example in the docs in this notebook.
    

In [ ]:
# Practice Here

#### Replacing Values
* [Relevanting Reading](https://www.practicalpythonfordatascience.com/03_cleaning_data#recoding-column-values).
* Step 1: Filter out for the rows that <b>didn't</b> merge. Find the unique values of the `project_name` column using `.unique()`
* Take a look at elements using 
    * Trailing white spaces
    * Capitalization
    * Spelling
    * Symbols

In [ ]:
m2.loc[m2._merge.isin(["left_only","right_only"])].project_name.unique()

* Step 2: Decide whether you want to rename the values in the left dataframe or the right one. 
* Step 3: The <b>keys</b>, are the values you want to replace. The <b>values</b>, are what you want to replace these values with. 

In [ ]:
new_names = {
    "main street muffin top ": "Main Street Muffin Top Revitalization",
    "Bunny Lane HOV+2 heaven": "Bunny Lane HOV+2 Haven",
    "Rainbow Rush hot  Lanes": "Rainbow Rush HOT Lanes",
}

* Step 4: Use your dictionary in `.replace()` to recode the values.

In [ ]:
projects_df.project_name = projects_df.project_name.replace(new_names)

#### Merge your dataframes again. This time it should work.


In [ ]:
final_m = pd.merge(projects_df, overall_scores_df, how="inner", on="project_name")

In [ ]:
final_m.shape

In [ ]:
final_m.head(1)

* Save this dataframe as a parquet to GCS under a new name

In [ ]:

final_m.to_parquet(f"{GCS_FILE_PATH}starter_kit_example_merge.parquet")

## Groupby
* You're done merging...Oh wait, that wasn't even part of your manager's request. You still need to aggregate. 
* The refresh your memory: by Caltrans District to find
    * Median overall score
    * Max overall score 
    * Min overall score
    * Number of unique projects
* There are many options Some are `groupby / agg`, `pivot_table`, `groupby / transform`
* <b>Resources</b>:[DDS Docs](https://docs.calitp.org/data-infra/analytics_new_analysts/01-data-analysis-intro.html#aggregating)
    * Use the space below and explore these tutorials. Then, apply your new knowledge to the prompt above.

In [ ]:
final_m["min_score"] = final_m.overall_score

In [ ]:
final_m["max_score"] = final_m.overall_score

In [ ]:
agg1 = (
    final_m.groupby(["ct_district"])
    .agg(
        {
            "overall_score": "median",
            "min_score": "min",
            "max_score": "max",
            "project_name": "nunique",
        }
    )
    .reset_index()
)

In [ ]:
agg1 = agg1.rename(
    columns={"overall_score": "median_score", "project_name": "n_projects"}
)

In [ ]:
agg1

## Visualizing 
* You're done aggregating, but the dataframe looks objectively plain.
* Unfortunately in the world of data, looks <b>do</b> matter. 
* Let's explore a couple of ways to present your data.

### Styling a Dataframe
* `pandas` has quite a few options that allow you to style your dataframe.
* [This tutorial](https://betterdatascience.com/style-pandas-dataframes/) offers some great ways to jazz up your dataframe.
* You can always read the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html) for more ideas.
* Some ideas:
    * Change the font
    * Turn off the index
    * Use colors to indicate low-high values
    * Change the alignment of the values

### Altair
* While a table is great, sometimes the stakeholder prefers a chart. 
* Our preferred visualization library is `Altair`, although there are other options.
    * Their docs page is [here](https://altair-viz.github.io/).
* The code to create a simple bar chart goes something like this. 
    * `alt.Chart(source).mark_bar().encode(x='a',y='b')`
    * `source` is the dataframe you want to use for your chart.
    * `x` denotes the column you are plotting on the X-axis. Make sure your column name has quotation marks around it. 
    * `y` denotes the column you are plotting on the Y-axis. 
* <b>Make your first chart below.</b>

In [ ]:
alt.Chart(agg1).mark_bar().encode(x="ct_district", y="n_projects")

#### Customizing
* `altair` offers an endless ways to amp up the personality of your chart.
* Additionally, the chart above without a title and legend is a data visualization "taboo" and the dull blue is uninspiring. 

##### Add a title
* You can do so within `.Chart()`

In [ ]:
alt.Chart(agg1, title="your_title_here").mark_bar().encode(
    x="ct_district", y="n_projects"
)

### Different Charts
* If you want something that isn't a bar chart, simply swap out `.mark_bar()` for `.mark_line` or `mark_circle`.


In [ ]:
alt.Chart(agg1, title="your_title_here").mark_circle().encode(
    x="ct_district", y="n_projects"
)

#### Add some color/DDS's Python Library
* We have some default color palettes that are already in our [internal library of functions](https://docs.calitp.org/data-infra/analytics_tools/python_libraries.html#calitp-data-analysis).

In [ ]:
# Import the color palettes like so  
from calitp_data_analysis import calitp_color_palette

* To see what is inside a module,  put two question marks behind it.
* From here, you can choose another color palette.

In [ ]:
calitp_color_palette??

In [ ]:
alt.Chart(agg1, title="your_title_here").mark_bar().encode(
    x="ct_district",
    y="n_projects",
    color=alt.Color(
        "n_projects",  # This is the column you want the color of your bar to be based on
        title="legend_title_here",  # This is the legend of your title
        scale=alt.Scale(
            range=calitp_color_palette.CALITP_DIVERGING_COLORS # This is where you can customize the colors,
        ),  
    ),
)

#### Adjusting the Axis
* Sometimes, we want to adjust the axis to have a min and max value.
* You do so using the `scale=alt.Scale(domain=[min_value, max_value]))` argument behind the X and Y axis.
* `alt.X()` and `alt.Y` gives you many more customization options.

In [ ]:
alt.Chart(agg1, title="your_title_here").mark_bar().encode(
    x=alt.X("ct_district", scale=alt.Scale(domain=[1, 12])),
    y=alt.Y("n_projects", scale=alt.Scale(domain=[0, 10])),
    color=alt.Color(
        "n_projects",
        title="legend_title_here",
        scale=alt.Scale(range=calitp_color_palette.CALITP_DIVERGING_COLORS),
    ),
)

### Finishing Touches 
* `.properties(width=400, height=250)` adjusts the size of your chart. 
* `tooltip=[columns you want]` gives you additional details on the columns you specify when you hover over each bar/circle/etc.
* `.mark_bar(size=30)` adjusts the size of the bar/circle/etc.

In [ ]:
alt.Chart(agg1, title="your_title_here").mark_bar(size = 10).encode(
    x=alt.X("ct_district", scale=alt.Scale(domain=[1, 12])),
    y=alt.Y("n_projects", scale=alt.Scale(domain=[0, 10])),
    color=alt.Color(
        "n_projects",
        title="legend_title_here",
        scale=alt.Scale(range=calitp_color_palette.CALITP_DIVERGING_COLORS),
    ),
    tooltip=["ct_district", "n_projects"]
).properties(width=400, height=250)

### We have only visualized one column of data. 
* We have only visualized one column of data, but we have a couple of columns above. 
* Make a few other charts in different styles. Altair's [gallery](https://altair-viz.github.io/gallery/index.html) is a great resource for inspiration.